In [3]:
from global_utils.utils import gen_dataset

music_path = "./Data/music/music_only/"
voice_path = "./Data/voice/"

data_path = "./data_path/"
gen_dataset(
    target_dir=data_path, music_dir=music_path, voice_dir=voice_path,
    voice_amp_ratio=0.3, train_test_split=0.3
)

Processing... [|] : [005/006]

[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


Processing... [-] : [006/006]	  Done


In [6]:
from global_utils.utils import TrainGenerator, ValidGenerator

train_path = "./data_path/train_data/"
valid_path = "./data_path/test_data/"

train_generator = TrainGenerator(train_path, bulk_num=2, sample_dur=2, sample_rate=110_000)
valid_generator = ValidGenerator(valid_path, bulk_num=2, sample_dur=2, sample_rate=110_000)

print(train_generator.input_shape, valid_generator.input_shape)

(960, 832, 2) (960, 832, 2)


In [7]:
from global_utils.utils import gen_unet

temp_model = gen_unet(train_generator.input_shape)
temp_model.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 960, 832, 2  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 480, 416, 32  1632        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 480, 416, 32  128        ['conv2d[0][0]']                 
 alization)  

In [8]:
from keras.callbacks import EarlyStopping

temp_model.compile(optimizer="adam", loss="mae", metrics=["mae", "mse"])

patience = 5
callback = EarlyStopping(patience=patience, verbose=True, restore_best_weights=True)

In [ ]:
model_path = "./model_save"
history = temp_model.fit(x=train_generator, epochs=patience * 10, validation_data=valid_generator)
temp_model.save(model_path + "checkpoint.h5")